In [1]:
from pathlib import Path
import os

yr = 2024
d = 12

inp_path = os.path.join(Path(os.path.abspath("")).parents[2], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))

with open(inp_path, 'r') as file:
    inp = file.readlines()

In [2]:
def format_input(inp):
  inp = [x.strip() for x in inp]
  return {(i,j): inp[i][j] for i in range(len(inp)) for j in range(len(inp[i]))}

In [3]:
import numpy as np

dirs = {(-1, 0), (1, 0), (0, -1), (0, 1)}

def add_dir(loc, dir):
  return (loc[0] + dir[0], loc[1]+dir[1])


class Region:

  def __init__(self):
    self.locs = set()
    self.val = None
    self.fences = 0
    self.area = 0

  def contains_square(self, loc, val):
    return self.val == val and any([any([add_dir(loc, dir) == loc_ for dir in dirs]) for loc_ in self.locs])

  def add_square(self, loc, val, fence):
    assert(self.contains_square(loc, val) or len(self.locs)==0)
    self.locs.add(loc)
    self.val = val
    self.fences += fence
    self.area += 1
  
  def __add__(self, other):
    assert(self.val==other.val)
    new = Region()
    new.locs = (self.locs).union(other.locs)
    new.val = self.val
    new.fences = self.fences + other.fences
    new.area = self.area + other.area
    return new


  def __str__(self):
    return f'Locs: {self.locs}\nVal: {self.val}\nArea: {self.area}\nFences: {self.fences}'
  
  def __repr__(self):
    return str(self)


def fence_fill(m):
  from functools import reduce
  regions = set()
  for k, v in m.items():
    cur_fence = sum([m.get(add_dir(k, dir), None)!=v for dir in dirs])
    potential_regions = [r for r in regions if r.contains_square(k,v)]
    if len(potential_regions) != 0:
      for pr in potential_regions:
        regions.remove(pr)
        cur_region = reduce(lambda x, y: x+y, potential_regions)
    else:
      cur_region = Region()
    cur_region.add_square(k, v, cur_fence)
    if cur_region not in regions:
      regions.add(cur_region)
  return regions





outer_corner_pat_base = np.array([[None, False, None], 
                                  [False, True, True], 
                                  [None, True, None]])

outer_corner_pats = [np.rot90(outer_corner_pat_base, k=i) for i in range(4)]


inner_corner_pat_base = np.array([[None, True, False], 
                                  [None, True, True], 
                                  [None, None, None]])


inner_corner_pats = [np.rot90(inner_corner_pat_base, k=i) for i in range(4)]

single_corner_pats = outer_corner_pats + inner_corner_pats



double_corner_pat_base = np.array([[None, False, None],
                                  [True, True, False], 
                                  [None, False, None]])


double_corner_pats = [np.rot90(double_corner_pat_base, k=i) for i in range(4)]


quadruple_corner_pat = np.array([[None, False, None],
                                  [False, True, False], 
                                  [None, False, None]])


def check_corner_pat(cur_loc, locs, pat):
  dirs = [[(-1,-1), (-1,0), (-1,1)], 
          [(0,-1), (0,0), (0,1)], 
          [(1,-1), (1,0), (1,1)]]
  for i in range(len(pat)):
    for j in range(len(pat[i])):
      if pat[i][j] != None and (pat[i][j]!=(add_dir(cur_loc, dirs[i][j]) in locs)):
        return False
  return True


def count_edges(region):
  count = 0
  for loc in region.locs:
    for corner_pat in single_corner_pats:
      count += check_corner_pat(loc, region.locs, corner_pat)
    for double_corner_pat in double_corner_pats:
      count += check_corner_pat(loc, region.locs, double_corner_pat)*2
    count += check_corner_pat(loc, region.locs, quadruple_corner_pat)*4
  return count

def sum_fences(regions):
  return sum([r.area*r.fences for r in regions])

def sum_edges(regions):
  return sum([r.area*count_edges(r) for r in regions])

In [4]:
import time

t = time.time()

formatted_input = format_input(inp)

print(sum_fences(fence_fill(formatted_input)))
print(sum_edges(fence_fill(formatted_input)))

print('\nRUNTIME: ', time.time()-t)

1431440
869070

RUNTIME:  22.756101608276367
